In [1]:
# Using a loop and the Pushift PRAW API to gather posts from r/stocks

In [2]:
!pip install praw

In [3]:
import pandas as pd
import requests
import time
import praw

In [4]:
# authentication information:

In [5]:
reddit = praw.Reddit(
    client_id="Jef0auDOdnMZ9vD6zlcWRg",
    client_secret="XqXCuNc590taBMgauwFmMBpDqvZp0A",
    user_agent="poppaearedditapilink",
    redirect_uri="http://localhost:8080",
)
print(reddit.auth.url(["identity"], "...", "permanent"))

https://www.reddit.com/api/v1/authorize?client_id=Jef0auDOdnMZ9vD6zlcWRg&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


In [6]:
def submissions_pushshift_praw(subreddit, start=None, end=None, limit=10000, extra_query=""):

    # creating an array to contain submissions
    posts = [] 
    


    # preparing api link with {} for inputs specified later
    api = ('https://api.pushshift.io/reddit/submission/search/?subreddit={}&after={}&before={}&sort_type=created_utc&sort=asc&limit={}&q={}')
    api = api.format(subreddit, start, end, limit, extra_query)

    # Using the api to fetch data in JSON form
    data = requests.get(api)
    json_data = data.json()['data']

    # Using a for loop to iterate over each submission, convert them to a praw_submission object, and append them to posts
    for submission in json_data:
        # Take the ID, fetch the PRAW submission object, and append to our list
        praw_submission = reddit.submission(id=submission['id'])
        posts.append(praw_submission)

    return posts


In [7]:
start_date = 1640304000 # Christmas Eve - 00:00:00
end_date = 1640390400  # Christmas Day 00:00:00

In [8]:
df = pd.DataFrame(columns=['title', 'body', 'created'])

In [9]:
# checking that the start date is infact before the end date, as a requirement for continuing to run the body of the loop
while start_date < end_date:
    # setting input values for our earlier url
    S = submissions_pushshift_praw(subreddit='stocks', start=start_date, end=end_date)
    # using a for loop over each post:
    for post in S:
        # stopping the loop from fetching deleted or removed posts
        try:
            if post.selftext != '[removed]' and post.selftext != '[deleted]':


                    df = df.append(
                        {'title': post.title,
                         'body': post.selftext,
                         'created': post.created}, ignore_index=True)
        except:
            # telling the loop to continue regardless of errors
            next()

    # telling the loop when to stop
    if len(S) < 100:
        break
    # making sure posts pull from the next earliest date value
    start_date = df['created'].max()
    print(start_date)

In [10]:
df['created'] = pd.to_datetime(df['created'],unit='s') # converting out of unix times

In [11]:
df

,title,body,created
0,How do you deal with frustration of missing ou...,I took a gamble on monday by risking 230$ on a...,2021-12-24 00:30:31
1,"Hypothetical: China begins invasion of Taiwan,...",There are two schools of thought on this:\n\n*...,2021-12-24 01:22:59
2,Favorite trade signal analysts to follow?,I am currently looking for people who follow t...,2021-12-24 01:24:06
3,Tencent to Hand $16 Billion Stake in JD.com to...,Internet giant Tencent Holdings Ltd. is planni...,2021-12-24 03:45:27
4,How do invest into the s&p 500?,Can i invest like 10 dollars into It or do I h...,2021-12-24 04:08:16
5,Thoughts about Walmart WMT fourth quarter,"With the fourth quarter ending December 31,\nj...",2021-12-24 04:41:55
6,Who is the best CEO right now and which CEO sh...,And I don't mean the charismatic and messianic...,2021-12-24 06:06:30
7,2021 Lookback : Low Profile stocks that made b...,What are your favorite lesser known stocks tha...,2021-12-24 10:23:47
8,r/Stocks Daily Discussion & Fundamentals Frida...,"This is the daily discussion, so anything stoc...",2021-12-24 10:30:11
9,r/Stocks Daily Thread on Meme Stocks Friday - ...,The meme stock scheduled posts will run Mon to...,2021-12-24 12:00:18


In [12]:
df["text"] = df["title"] + df["body"]

In [13]:
df = df.drop(["title", "body"], axis = 1)

In [14]:
!pip install TextBlob
from textblob import TextBlob

# data pre processing from TextBlob
def preprocess(text):
    text = text.str.replace("(<br/>)", "")
    text = text.str.replace('(<a).*(>).*(</a>)', '')
    text = text.str.replace('(&amp)', '')
    text = text.str.replace('(&gt)', '')
    text = text.str.replace('(&lt)', '')
    text = text.str.replace('(\xa0)', ' ')  
    return text
df['text'] = preprocess(df['text'])

df['polarity'] = df['text'].map(lambda text: TextBlob(text).sentiment.polarity)
df['review_len'] = df['text'].astype(str).apply(len)
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

C:\Users\poppa\AppData\Local\Temp/ipykernel_23704/2426943112.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace("(<br/>)", "")
C:\Users\poppa\AppData\Local\Temp/ipykernel_23704/2426943112.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('(<a).*(>).*(</a>)', '')
C:\Users\poppa\AppData\Local\Temp/ipykernel_23704/2426943112.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('(&amp)', '')
C:\Users\poppa\AppData\Local\Temp/ipykernel_23704/2426943112.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('(&gt)', '')
C:\Users\poppa\AppData\Local\Temp/ipykernel_23704/2426943112.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.repla

In [15]:
df = df.drop(["review_len", "word_count", "text"], axis = 1)

In [16]:
df

,created,polarity
0,2021-12-24 00:30:31,0.124306
1,2021-12-24 01:22:59,0.053968
2,2021-12-24 01:24:06,0.258333
3,2021-12-24 03:45:27,0.188095
4,2021-12-24 04:08:16,0.162121
5,2021-12-24 04:41:55,0.160000
6,2021-12-24 06:06:30,0.399643
7,2021-12-24 10:23:47,0.130000
8,2021-12-24 10:30:11,0.019213
9,2021-12-24 12:00:18,0.218869


In [17]:
# df.to_csv(r'C:\Users\poppa\OneDrive - University of Bristol\3rd\data science\project\r_stocks_data\current\month_polarity\compile\nov3.csv', index = False)

In [18]:
# Sources
# TextBlob Docs
# Pushift PRAW Docs
# David Little: 4-Step Reddit vs. NY Stock Exchange Sentiment Analysis

In [19]:
df


,created,polarity
0,2021-12-24 00:30:31,0.124306
1,2021-12-24 01:22:59,0.053968
2,2021-12-24 01:24:06,0.258333
3,2021-12-24 03:45:27,0.188095
4,2021-12-24 04:08:16,0.162121
5,2021-12-24 04:41:55,0.160000
6,2021-12-24 06:06:30,0.399643
7,2021-12-24 10:23:47,0.130000
8,2021-12-24 10:30:11,0.019213
9,2021-12-24 12:00:18,0.218869
